#### Dependencies:

In [1]:
import os
import torch #pip install torch
import numpy as np #pip install numpy
#import pandas as pd 
import pickle
import xgboost as xgb #pip install xgboost
import tensorflow as tf #pip install tensorflow
from tensorflow.keras import layers
#from rdkit import Chem #conda install -c rdkit rdkit
#from rdkit.Chem import Crippen 
#from rdkit.Chem import Descriptors
import shutil
import warnings
import esm #pip install fair-esm

2025-02-15 18:52:14.051434: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /n/sw/helmod-rocky8/apps/Core/gcc/13.2.0-fasrc01/lib64:/n/sw/helmod-rocky8/apps/Core/gcc/13.2.0-fasrc01/lib:/n/sw/helmod-rocky8/apps/Core/mpc/1.3.1-fasrc02/lib64:/n/sw/helmod-rocky8/apps/Core/mpfr/4.2.1-fasrc01/lib64:/n/sw/helmod-rocky8/apps/Core/gmp/6.3.0-fasrc01/lib64
2025-02-15 18:52:14.051458: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from KM_prediction import *
import os
from os.path import join 
#import torch
CURRENT_DIR = os.getcwd()

Before using KM prediction for the first time, we have to download the ESM-1b model:

In [3]:
import esm
model, alphabet = torch.hub.load("facebookresearch/esm:v0.4.0", "esm1b_t33_650M_UR50S")

Using cache found in /n/home00/stephentsou/.cache/torch/hub/facebookresearch_esm_v0.4.0


Example for calculating KM prediction. You can enter multiple enyzmes and substrates. Enzyme 1 will be paired with substrate 1, enzyme 2 will be paired with substrate 2 and so on...

Enter enzymes as amino acid sequences and substrate either as SMILES strings, KEGG Compound IDs, or InChI strings.

In [4]:
from KM_prediction import *
#this is for dhfr uncomment
#substrate_list = ["InChI=1S/C19H23N7O6/c20-19-25-15-14(17(30)26-19)23-11(8-22-15)7-21-10-3-1-9(2-4-10)16(29)24-12(18(31)32)5-6-13(27)28/h1-4,11-12,21,23H,5-8H2,(H,24,29)(H,27,28)(H,31,32)(H4,20,22,25,26,30)/t11-,12-/m0/s1;InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p+1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1",]

substrate_list = ["InChI=1S/C19H21N7O6/c20-19-25-15-14(17(30)26-19)23-11(8-22-15)7-21-10-3-1-9(2-4-10)16(29)24-12(18(31)32)5-6-13(27)28/h1-4,12,21H,5-8H2,(H,24,29)(H,27,28)(H,31,32)(H4,20,22,25,26,30)/t12-/m0/s1",]

#products = ["InChI=1S/C6H5NO4/c8-5-2-1-4(7(10)11)3-6(5)9/h1-3,8-9H;InChI=1S/CH2O2/c2-1-3/h1H,(H,2,3);InChI=1S/H2O/h1H2"]

#dihydrofolate acid, NADPH uncomment below
#products = ["InChI=1S/C19H21N7O6/c20-19-25-15-14(17(30)26-19)23-11(8-22-15)7-21-10-3-1-9(2-4-10)16(29)24-12(18(31)32)5-6-13(27)28/h1-4,12,21H,5-8H2,(H,24,29)(H,27,28)(H,31,32)(H4,20,22,25,26,30)/t12-/m0/s1;InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(44-46(33,34)35)14(30)11(43-21)6-41-48(38,39)45-47(36,37)40-5-10-13(29)15(31)20(42-10)27-3-1-2-9(4-27)18(23)32/h1,3-4,7-8,10-11,13-16,20-21,29-31H,2,5-6H2,(H2,23,32)(H,36,37)(H,38,39)(H2,22,24,25)(H2,33,34,35)/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1;InChI=1S/p+1"]

#H+
#;1S/p+1


#enzymes = ["MKYFPLFPTLVFAARVVAFPAYASLAGLSQQELDAIIPTLEAREPGLPPGPLENSSAKLVNDEAHPWKPLRPGDIRGPCPGLNTLASHGYLPRNGVATPVQIINAVQEGLNFDNQAAVFATYAAHLVDGNLITDLLSIGRKTRLTGPDPPPPASVGGLNEHGTFEGDASMTRGDAFFGNNHDFNETLFEQLVDYSNRFGGGKYNLTVAGELRFKRIQDSIATNPNFSFVDFRFFTAYGETTFPANLFVDGRRDDGQLDMDAARSFFQFSRMPDDFFRAPSPRSGTGVEVVIQAHPMQPGRNVGKINSYTVDPTSSDFSTPCLMYEKFVNITVKSLYPNPTVQLRKALNTNLDFFFQGVAAGCTQVFPYGRD"]

#DHFR
#enzymes = ["MVGSLNCIVAVSQNMGIGKNGDLPWPPLRNEFRYFQRMTTTSSVEGKQNLVIMGKKTWFSIPEKNRPLKGRINLVLSRELKEPPQGAHFLSRSLDDALKLTEQPELANKVDMVWIVGGSSVYKEAMNHPGHLKLFVTRIMQDFESDTFFPEIDLEKYKLLPEYPGVLSDVQEEKGIKYKFEVYEKND"]

#1
#enzyme_list = ["MISFIFAMDANRLIGKDNDLPWHLPNDLAYFKKITSGHSIIMGRKTFESIGRPLPNRKNIVVTSAPDSEFQGCTVVSSLKDVLDICSGPEECFVIGGAQLYTDLFPYADRLYMTKIHHEFEGDRHFPEFDESNWKLVSSEQGTKDEKNPYDYEFLMYEKKNSSKVGGF"]
#2
#enzyme_list = ["MTFSLIVATTLNSVIGKDNQMPWHLPADLAWFRQNTTGKPVIMGRKTFESIGRPLPKRTNIVLSRQPFKHEGVVWKNSLESAVNFVRDFDEIMLIGGGELFKQYLPKADKLYLTQIQTELDGDTFFPQLNWEEWKIEFDEYHKADEQNRYDCRSLILTRK"]
#3
#enzyme_list = ["MTFSLIVAMTKNHVIGKDNQMPWHLPADLAWFRQNTTGKPVIMGRKTFESIGRPLPKRTNLVLSRQPYLHEGVIWQNSLESAVDFVKDSPEILLIGGGELFKQYLHKADKLYLTEIQTELEGDTFFPEIVWQDWQIEYEAWHQADEKNPYDCRFLILSRNTKNE"]
#4
#enzyme_list = ["MIIGIWAEDEAGLIGEADKMPWSLPAEQQHFKETTMNQVILMGRKTFEGMNKRVLPGRISIILTRDETYQSDNEKVLIMHSPKEVLDWYHKQNKDLFITGGAEILALFESELELLYRTVVHEKFKGDTYFPSTFDFGRFKLVSEKFHDKDERNSYTFTIKKYEKVKQP"]
#5
#enzyme_list = ["MLSIIAALNENYVIGNENKLIWHISDDLKRFKKLTTGKTIIMGRKTFESLPGVLPNRKHIVITKNLNYSRENVSIVHSIDEIIELKDTTEENFIIGGGEIYRALIPYCSILYLTKVHSNQTGDAFFPKFDENDYSIIATEKHGDYDFVTFKRR"]
#6
#enzyme_list = ["MIIFVWAQDRAGNIGKDNKMPWHLPGDLQFFKKTTTGKTLVMGRKTYESLGKALPNRKTIVLTRDNELQLDDAEILHSRDEVLALAETGEPIYVVGGAEIYRLFMDVADKLIVTKIDAEFDADTAFPEVDWENFSEVAKEPHEKDEKNKYNYTFYTYERN"]
#7
#enzyme_list = ["MPMPATPLLSLVVAYSTNGVIGRDNALPWKLPGDLAHFKRTTLGHPIIMGRNTWESLGRPLPGRTNIVITRNPAYGAAGAVVVGSLDAAIQACGDAAVACVIGGAQIYAQALPLAQRVVATEVHAEIEGDAFFPPLPAGQWRETQRAAQPAENGLRYDFVEYERVAG"]
#8
#enzyme_list = ["MIISLIAALAADRVIGMENAMPWHLPADLAWFKRNTLNKPVIMGRKTFESIGRPLPGRLNIVISSQPGTDERVTWAASIEEALAFAGNAEEVMVMGGGRVYKQFLDRANRMYLTHIDAEVGGDTHFPDYEPDEWESVFSEFHDADEANSHSYCFEILERR"]
#9
#enzyme_list = ["MTRPPDISLIVAMARNRVIGADNAMPWHLSADLKRFRALTLGKPVLMGRKTHESIGRVLPGRQNIILTRSPDFHAPGCTIVHDLGELATVCSGAPELMVIGGSAVYEALLPRARRLYLTLIDHDYPGDTYFPEFAAADWREISREDVRNDPDFPWPYSYLVLERASPEARCQ"]
#10
#enzyme_list = ["MSGTGQPRVSVIAALAKNRVIGIENRLPWRLPEDLAHFKALTLDHPVLMGRKTFESLGRPLPRRTNVVITRNADYRPDGCLVAASMPAAIALCQGADEIFFIGGAELYAQAIPLADRLYLTEVDAEPEGDAWFPEFDRAAFREISRASHVGEKGDVLRFDFVVYERR"]
#11
#enzyme_list = ["MAQLTLIVAYARNRVIGRDNTLPWRLPGDLAHFKRSTLGHPIVMGRNTWESLGRPLPGRKNIVVSRNPDYRAEGAIVVPDLQSALKMAEADEVFVIGGAQIYSQALPLATRIIATEIQANVAGDAYFPPLAQTEWRETSRQPQPEENGYHYDFVVYERNR"]
#12
#enzyme_list = ["MTKIKMVWAEDRQHAIGKDGGIPWHMPDDLKLFRDETVNTLMIMGRPTWLSIGRPLPKRTTVVMTRQEDWTPSYPEVKVIHSIEEAKNLIAKEERDITIAGGAAVYREFMPYATDLVITRVDGVIDGDTFVDEVDLTQFQLKSSEPHAKDDNHDYAFVVERYERI"]
#13
#enzyme_list = ["MMISMIAAMAHDRVIGLDNQMPWHLPADLAHFKRVTLGKPVLMGRKTFESIGRPLPGRRNLVISRNPDYRADGVEVIDSVDAALALLAGSDVAELMVIGGGHLYGQLLPRADRLYLTRIDLAVEGDTRFPAFDEGDWSCIERESHQPDEKNPHSYCFETWQRR"]
#14
#enzyme_list = ["MRIAMIAAMANNRVIGKDNQMPWHLPEDLKHFKAMTLGKPVVMGRKTFESIGRPLPGRHNIVISRQSDYSHEGISLVSSFEEAVALAGDCEELVVIGGGQLYQTLISQADKLYLTEIALEVEGDTHFPAWDDGSWETVESTEHVSDKGLQYRFITLVKKC"]
#16
#enzyme_list = ["MISMIWAMGRNNALGCKNRMPWYIPADFAYFKKVTMGKPVIMGRKTFESIGKPLPGRKNIVITRDTGYDPQGCIVVNSIEKAMEYTEEKEVFIIGGAEIYKEFLPIADRLYITLIEKEFEADAFFPEIDYSKWKQISCETGIKDEKNPYEYKWLVYERVKQ"]
#17
#enzyme_list = ["MTKEMIAIWAEDEAGLIGVDGKLPWYLPRELQHFKETTLNQAILMGRVTFDGMNRRLLPNRQTLVMTRDVNYQVDGVLTMTSVEEVLDWYHAQEKTLYVIGGSKVLEAFDGYFNRVIKTVVHHQFEGDTYRPKLDLSRFREESQTFYPKDANNPYDFTVTVLKHQ"]
#19
#enzyme_list = ["MINLIWAQDANGNIGKDNQMPWRLPADLAYFKKQTTGKTIVMGRKTYDSLGKALPNRENIVLTRDPELTLADASVVHTKADILKRAEKEAIFIIGGAQIYALFADDADRLYVTKIAATFDADTAFPVLDWSAFRLVQETPGHIDEKNKYPHAFLVYEREQEIQ"]
#20
#enzyme_list = ["MIISLIAALAADRVIGMENAMPWHLPADLAWFKRNTLNKPVIMGRKTFESIGRPLPGRLNIVISSQPGADSRVSWATSIDEALALAGEVEEVMVMGGGHVYSQFLDRAHRMYLTHIDAEVGGDTHFPDYEPDEWETSFSEFHDADEANSHSYCFEVLERR"]
#21
#enzyme_list = ["MHKMAEKNLIWAQTLDGTIAIDGHIPWHQKADLQFFKQSTIHEAALMGRHTMASFHGRPLPERLNLVLTRNHDLEVPEGFQKVYSVAEAEKVADNANLKLQVIGGKPIYESFMDTADTLYVTYLQTDFSGDVKMAPVDLTVWQGEVIDQGPADADNDYDWRLVKYTRQ"]
#22
#enzyme_list = ["MTTISLIAAIDESGGLGINNQLLCHLPADLQHFKSITMGKPIIMGRKTFDSIGKPLPGRLNIVLSRSLSFIDGVHVLDSLEKAIEYTAGHDEIMIIGGAVLFNDAIDKASSLYITHIHHQFAADVFFPKIDKSVWHCQNEQFRHQDEKNKHDMTFCFYERKH"]
#23
#enzyme_list = ["MKLSLMVAISKNGVIGNGPDIPWSAKGEQLLFKAITYNQWLLVGRKTFESMGALPNRKYAVVTRSSFTSDNENVLIFPSIKDALTNLKKITDHVIVSGGGEIYKSLIDQVDTLHISTIDIEPEGDVYFPEIPSNFRPVFTQDFASNINYSYQIWQKG"]
#26
#enzyme_list = ["MKLTLIVAFDRNNAIGRDNDLPWKLPDDLKRFKALTLGKPILMGRKTAQSLGRALPERLNLVLTRSGQVPFEGMHAVASVEQAIERAAQDGAQELCVIGGGEVYRLTMQRADLLAVTEVDTVVDGADTHFPPIDPALWEAVQRESHAADARHAFAFDYVDYRRR"]
#27
#enzyme_list = ["MNTSLPLSLIAAYAENRVIGIDNSMPWHLPGDFKYFKATTLGKPIIMGRKTWDSLGRPLPGRLNIVVSRQPGLELAGAEVFASLEEALVRAEQWAREQGVGELMLIGGAQLYGQALEKGLVSRMYLTRVELAPEGDAWFPAFDEGQWRLTSSEAQAEEGKPAYHFEVWDKG"]
#30
#enzyme_list = ["MIISFIVAMDENHLIGANRRLPWHLPADLKYFRRMTMGKPMLMGRSTHEAIGRPLPGRHNIVVTHNPDYKAPGCTVVHTVEAGLQAAGEAEEVTVIGGASLYEQLLPQARRIYLTLVRGHFQGDTWFPAFDPGTWVEVWREDYDPDPQNPYPYSFIRLEREERKK"]
#31
#enzyme_list = ["MISMIWAEDLRHGIGKQQSIPWHIPGDMAFFKKTTSGNTVVMGRKTFDSIGKALPNRKNIVLSHHPTSLPDSVVGVGSLSELQAIFETHPNEKFYIIGGSHLYNALLSQADELLITRIQKDYQCEVLAPDITQNEFKLASSVEHESTERNPAYTFETWIRI"]
#32
#enzyme_list = ["MISIIVAIAKDNVIGKDNKLLWHISEDLKRFKKITTGKKMIMGRKTFESLPGILPNREHIVLTRDNNFNVDSDKVTIEHDFNSVLQRYSECEDEVFVIGGAEIYKQFLPYAKKLYLTKVDEEFEGDTYFPGINYSNYNTEYTSEKFIDEKNGLHYTFVNLIQA"]
#33
#enzyme_list = ["MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVVMGRHTWESIGRPLPGRKNIVISSQPGTDDRVQWVKSVDEAIAACGDAPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADEQNSHSYCFEILERR"]
#35
#enzyme_list = ["MIISLIAALSADRVIGLDNAMPWHLSADLAWFKRNTLAKPVIMGRKTYESIGRPLPGRHNIVISRQAGCDDRVSWVATPEAALTAAADADEVMIIGGGTLYEQFLPQASRLYLTHVDADVAGDTYFPDYAPDEWQTCFSEFHEADSDNSHAYCFEILDRR"]
#36
#enzyme_list = ["MKIGLVAALTRNQVIGRYNALPWSLPADLQRFKKITMGKPIIMGRKTYDSIGRPLPGRKNIVISRNPDFHADGVTVVDSLDMALEAADQAPEVMVIGGANIYYQFLPRADRLYFTLVHTTIDDGDAFFPAYNRRDWRLVIEENHPADADNPFPYSFMTWQRITPPKNQSDTPEAHHI"]
#37
#enzyme_list = ["MAPVFFSEPFNMISLIAALAADRIIGMENAMPWDLPADLAWFKRNTLKKPVIMGRLTFESIGRPLPGRLNIVVSSKPGSAEGVTWVTSLDDAISAAGEAEEIMVIGGGRIYEQMLARADRLYLTHIDAEVEGDTQFPDYEPDEWHSTFSEFHDADEQNSHSYCFEILERRR"]
#38
#enzyme_list = ["MITSIWAQSKNNVIGRNNRLPWSLPDDLKFFRQETKNKAVVMGRKTYESFGSKALPKRLNIILTSNMDFKSNDPEVKIVHSPIEAVNTAKEQHLPLYVIGGASVYESFMNMADRLLITLVDANIKGDTFAPNFSEKSFSLVSQCHHAQDKKHAYSFDFLTYERKNKSDQV"]
#39
#enzyme_list = ["MTTLTLIVARARNGVIGRDNQLPWKLPEDLAFFKRTTMGAPIVMGRRTHESIGRPLPGRRNIVVTRDAARRFDGCDTVTSLDGALALAARDGAAEAFLIGGAQLYAEGLRHADKLIVTEIDQDFEGDASFPAPDPAQWEAVSRDAHRAAAPNDFAYAFVVYRRKRAG"]
#41
#enzyme_list = ["MPKITLIAACAPDRCIGINNTMPWHLPEDFAFFKSYTLDKPVVMGRKTWESLPRKPLPGRRNIVISRQADYPAEGAETVGSLEEALALCAAAEEVIIMGGAQIYAQALPLASDLRITEVALDVQGDAFFPEFSPSEWREAARETHTSANGTGYAFVHYTRIR"]
#42
#enzyme_list = ["MMISMIAAMAENRIIGKDNQMPWHLPADFAWFKRCTLGKPVIMGRKTYQSIGRPLPGRHNIVISRDESLQIDGVDVVTSIDAALAKAGEVDEIMIIGGGSLYAACLPMANKLYITEINAELDGDTQFPAWGSDWQECYREHYPADEKNTYAMDFVIFERK"]
#43

enzyme_list = ["MSEARPFTLALVVAAAENDVIGREGKLPWRLKSDLKRFRRLTMGHPLIMGRKTFASIGKPLDGRDSVIVTRDAASVSPQPGVFVATSIKDALDIARARAAERRVAEAFVIGGAEIFALALPYADRIHLARVHASPSGDAYWQAPPAEEWQVVSREEWPANQTDEFSVTDLVLERSPRA"]


KM_predicton(substrate_list, enzyme_list)




#KM_predicton(substrate_list = ["InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p-1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1"], enzyme_list = ["MSIPETQKGVIFYESHGKLEYKDIPVPKPKANELLINVKYSGVCHTDLHAWHGDWPLPVKLPLVGGHEGAGVVVGMGENVKWKIGDYAGIKWLNGSCMACEYCELGNESNCPHADLSGYTHDGSFQQYATADAVQAAHIPQGTDLAQVAPILCAGITVYKALKSANLMAGHWVAISGAAGGLGSLAVQYAKAMGYRVLGIDGGEGKEELFRSIGGEVFIDFTKEKDIVGAVLKATDGGAHGVINVSVSEAAIEASTRYVRANGTTVLVGMPAGAKCCSDVFNQVVKSISIVGSYVGNRADTREALDFFARGLVKSPIKVVGLSTLPEIYEKMEKGQIVGRYVVDTSK"])

Step 1/3: Calculating numerical representations for all metabolites.
.....1(a) Calculating input matrices for Graph Neural Network
.....1(b) Calculating numerical metabolite representations using a Graph Neural Network


2025-02-15 18:55:14.202857: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2025-02-15 18:55:14.345970: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:ca:00.0 name: NVIDIA A100-SXM4-40GB MIG 3g.20gb computeCapability: 8.0
coreClock: 1.41GHz coreCount: 42 deviceMemorySize: 19.50GiB deviceMemoryBandwidth: 724.20GiB/s
2025-02-15 18:55:14.348408: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /n/sw/helmod-rocky8/apps/Core/gcc/13.2.0-fasrc01/lib64:/n/sw/helmod-rocky8/apps/Core/gcc/13.2.0-fasrc01/lib:/n/sw/helmod-rocky8/apps/Core/mpc/1.3.1-fasrc02/lib64:/n/sw/helmod-rocky8/apps/Core/mpfr/4.2.1-fasrc01/lib64:/n/sw/helmod-rocky8/apps/Core/gmp/6.3.0-fasrc01/lib64
2025-02-15 18:55:14.358028: W tensorflow/

hello
(None, 50)
Step 2/3: Calculating numerical representations for all enzymes.
.....2(a) Loading ESM-1b model.
.....2(b) Calculating enzyme representations.
Step 3/3: Making predictions for KM.


,substrate,enzyme,GNN rep,enzyme rep,complete,KM [mM]
0,InChI=1S/C19H21N7O6/c20-19-25-15-14(17(30)26-1...,MSEARPFTLALVVAAAENDVIGREGKLPWRLKSDLKRFRRLTMGHP...,"[17.985392, 128.0979, 15.768364, 49.953487, 0....","[0.112979494, 0.1907936, -0.00805205, 0.046816...",True,0.009852
